In [2]:
import tensorflow as tf

x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

In [6]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)

42


In [7]:
sess.close()

In [8]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

In [12]:
result = 0

In [13]:
init = tf.global_variables_initializer()  # prepare an init node

with tf.Session() as sess:
    init.run()  # actually initialize all the variables
    result = f.eval()

In [14]:
result

42

In [15]:
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close()

42


In [16]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [18]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

In [19]:
x2.graph is graph

True

In [20]:
x2.graph is tf.get_default_graph()

False

In [21]:
#tf.reset_default_graph().

In [27]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3
with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


In [30]:
with tf.Session() as sess:
    y_val,z_val = sess.run([y,z])
    print(y_val)
    print(z_val)

10
15


In [31]:
import numpy as np
from sklearn.datasets import fetch_california_housing

In [34]:
housing = fetch_california_housing()
m,n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

In [35]:
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

In [38]:
#compute theta
#Theta = (XT · X)–1 · XT · y
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

In [39]:
with tf.Session() as sess:
    theta_value = theta.eval()

In [40]:
theta_value

array([[ -3.74636536e+01],
       [  4.35707688e-01],
       [  9.34202131e-03],
       [ -1.06593400e-01],
       [  6.43945396e-01],
       [ -4.25595226e-06],
       [ -3.77299124e-03],
       [ -4.26701427e-01],
       [ -4.40539479e-01]], dtype=float32)

In [42]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [45]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)

    best_theta = theta.eval()

Epoch 0 MSE = 8.32148
Epoch 100 MSE = 0.75015
Epoch 200 MSE = 0.60187
Epoch 300 MSE = 0.582656
Epoch 400 MSE = 0.569739
Epoch 500 MSE = 0.559892
Epoch 600 MSE = 0.552323
Epoch 700 MSE = 0.546471
Epoch 800 MSE = 0.541926
Epoch 900 MSE = 0.538375


In [46]:
gradients = tf.gradients(mse, [theta])[0]

In [47]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

In [48]:
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,
                                       momentum=0.9)

In [49]:
A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5

In [50]:
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict={A: [[4, 5, 6], [7, 8, 9]]})

In [51]:
print(B_val_1)

[[ 6.  7.  8.]]


In [52]:
print(B_val_2)

[[  9.  10.  11.]
 [ 12.  13.  14.]]


In [53]:
X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

In [54]:
batch_size = 100
n_batches = int(np.ceil(m / batch_size))